In [1]:


import os
import re
from scipy.io import loadmat
import mat73

from collections import Counter
import pandas as pd
import numpy as np
import mne

import seaborn as sns
import matplotlib.pyplot as plt

from utils import *
from connectivity_dynamics import *

In [2]:
TIMELINE_F = '/mnt/ernie_main/000_Data/SEEG/SEEG_Periictal/data/Extracted_Per_Event_Interictal/all_time_data_01042023_212306.csv'
SEEG_FOLDER = '/mnt/ernie_main/000_Data/SEEG/SEEG_Entire_EMU_Downloads/data/'

In [3]:
seeg_timeline_df = pd.read_csv(TIMELINE_F, delimiter="\t")
seeg_timeline_df.head(5)

,Type,Pat ID,Event Number (Matches EMU Final Report),Date (MM:DD:YYYY),Onset String (HH:MM:SS),Offset String (HH:MM:SS),Seizure Type (FAS; FIAS; FBTC; Non-electrographic; Subclinical; Unknown),FileIndex,onset_datetime,offset_datetime,onset_datetime_buffer,offset_datetime_buffer,FileName
0,File,Epat02,File1,09:14:2017,13:05:11,17:05:11,NaN,0,2017-09-14 13:05:11,2017-09-14 17:05:11,2017-09-14 13:06:11,2017-09-14 17:04:11,Epat02_09142017_13051100.EDF
1,File,Epat02,File2,09:14:2017,17:05:22,21:05:22,NaN,1,2017-09-14 17:05:22,2017-09-14 21:05:22,2017-09-14 17:06:22,2017-09-14 21:04:22,Epat02_09142017_17052200.EDF
2,File,Epat02,File3,09:14:2017,21:05:34,01:05:34,NaN,2,2017-09-14 21:05:34,2017-09-15 01:05:34,2017-09-14 21:06:34,2017-09-15 01:04:34,Epat02_09142017_21053400.EDF
3,File,Epat02,File4,09:15:2017,01:05:45,05:05:45,NaN,3,2017-09-15 01:05:45,2017-09-15 05:05:45,2017-09-15 01:06:45,2017-09-15 05:04:45,Epat02_09152017_01054500.EDF
4,File,Epat02,File5,09:15:2017,05:05:56,09:05:56,NaN,4,2017-09-15 05:05:56,2017-09-15 09:05:56,2017-09-15 05:06:56,2017-09-15 09:04:56,Epat02_09152017_05055600.EDF


In [4]:
label_f = '~/Ephys/Data/all_pats_bipole_soz_labels.csv'
label_df = pd.read_csv(label_f, header=None)
label_df.columns = ['subj','bipole','label']



label_df = split_bipole(label_df)

label_df.label = label_df.label.apply(map_label)
label_df['bipole'] = label_df.bipole.apply(lambda x: x.replace(" ", ""))

# Calculating Connectivity Between Channels

It seems my task is much easier than I previously thought. There are designated recordings for my 4 periods of interested already cleaned and labelled by Derek and Graham. Bless.

Goal for this section's exploration: calculate ISH across 4 periods for 1 patient.
1. Find 4 recordings for patient
1. Calc Connectivity (PDC) for each recording
1. Im show some of those recordings
1. calculate inward-outward strengh for 4 periods - ISH
1. BONUS: Redo 1-4 with cross spectral coherence connectivity from the French

## Using Precomputed PDC Peri-Ictally

In [5]:
pat = 'Epat02'
conn_f = "/mnt/ernie_main/Ghassan/ephys/data/connectivity/Epat02/Epat02_10_FIAS_imcoh.mat"
conn_obj = load_mat(conn_f)

ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: table, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: table, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32

In [6]:
## loading the PDC and labels
regions = [reg[0] for reg in conn_obj['pdc']['seizure']['bip_labels_used']]
regions = format_bipoles(regions)


In [7]:
#get labels of each patient's bipoles as SOZ, PZ, etc
pat_df = label_df[label_df.subj==pat]
contact_designation = pat_df.label.values
bipoles = pat_df.bipole.values
bip_dict = dict(zip(bipoles, contact_designation))
pat_conn_labels = np.array([bip_dict[b] for b in regions])

In [8]:

soz_inds, pz_inds, nz_inds = get_reg_inds(pat_conn_labels)

In [5]:
data_dir = '/mnt/ernie_main/Ghassan/ephys/data/connectivity/'
folders = glob.glob(os.path.join(data_dir, "*pat*"))
sub_ids = [os.path.basename(f) for f in folders]

In [6]:
tst_folders = folders[0:2]
tst_subj = sub_ids[0:2]

In [20]:
tst_df = agg_subjects(tst_subj, tst_folders, label_df)

ValueError: No objects to concatenate

In [18]:
[f for f in tst_folders if len(glob.glob(os.path.join(f, "*.mat"))) != 0 ]


['/mnt/ernie_main/Ghassan/ephys/data/connectivity/Epat02',
 '/mnt/ernie_main/Ghassan/ephys/data/connectivity/Epat03']

In [10]:
BANDS = ['delta', 'theta', 'alpha', 'beta','gamma_l', 'gamma_H']
PERIOD = ['inter','pre','ictal','post']


In [12]:

pdc_dict = get_conn_dict(conn_obj)

In [13]:
subj_id = pat


net_df = assemble_net_conn(subj_id,pdc_dict,soz_inds,pz_inds, nz_inds)

In [14]:
tst_df = assemble_file(subj_id, conn_f, pat_conn_labels)

ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: table, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: table, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32

In [3]:
pwd

'/home/ghassan/Ephys/Code/dynamic_ISH/code'

In [6]:
len(glob.glob('/home/ghassan/Ephys/Code/dynamic_ISH/code/*.pxy'))

0

In [2]:
os.path.basename("/test/str")

'str'